In [44]:
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("../data/speaker_dialogue.csv")
df

,speaker,dialogue,file,file_index,saha_page
0,MR PRIOR,May I proceed Mr Chairman?,../data/data_saha\amnesty_hearings\aliwal_nort...,0,https://sabctrc.saha.org.za/documents/amntrans...
1,CHAIRPERSON,Yes.,../data/data_saha\amnesty_hearings\aliwal_nort...,1,https://sabctrc.saha.org.za/documents/amntrans...
2,MR PRIOR,"Mr Chairman, the matter which proceeds this mo...",../data/data_saha\amnesty_hearings\aliwal_nort...,2,https://sabctrc.saha.org.za/documents/amntrans...
3,CHAIRPERSON,Thank you.,../data/data_saha\amnesty_hearings\aliwal_nort...,3,https://sabctrc.saha.org.za/documents/amntrans...
4,MR MBANDAZAYO,"Thank you Mr Chairman. Mbandazayo's my name, ...",../data/data_saha\amnesty_hearings\aliwal_nort...,4,https://sabctrc.saha.org.za/documents/amntrans...
...,...,...,...,...,...
675935,CHAIRPERSON,Joyce Seroke.,../data/data_saha\special_hearings\women's_hea...,36,https://sabctrc.saha.org.za/documents/special\...
675936,MS SEROKE,"Mrs Thobela, this is a painful story that you ...",../data/data_saha\special_hearings\women's_hea...,37,https://sabctrc.saha.org.za/documents/special\...
675937,LEPHINA ZODWA THOBELA,"No, I have had peace and quiet and after the r...",../data/data_saha\special_hearings\women's_hea...,38,https://sabctrc.saha.org.za/documents/special\...
675938,MS SEROKE,We thank you very much for having come forward...,../data/data_saha\special_hearings\women's_hea...,39,https://sabctrc.saha.org.za/documents/special\...


In [23]:
df.dtypes

speaker       object
dialogue      object
file          object
file_index     int64
saha_page     object
dtype: object

In [28]:
df.dialogue = df.dialogue.apply(str)

In [29]:
df.dtypes

speaker       object
dialogue      object
file          object
file_index     int64
saha_page     object
dtype: object

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [30]:
doc_embeddings =  model.encode(df["dialogue"])

In [ ]:
umap_proj = UMAP(n_neighbors=10,
                          min_dist=0.01,
                          metric='correlation').fit_transform(doc_embeddings)

In [ ]:
hdbscan_labels = hdbscan.HDBSCAN(min_samples=15, min_cluster_size=15).fit_predict(umap_proj)
print(len(set(hdbscan_labels)))

In [ ]:
# Apply coordinates
df["x"] = umap_proj[:, 0]
df["y"] = umap_proj[:, 1]
df["topic"] = hdbscan_labels
df.head(1)

In [31]:
doc_embeddings.shape

(675940, 384)

In [32]:
annoy_index = AnnoyIndex(384, metric='angular')

In [33]:
for idx, emb in enumerate(doc_embeddings):
    annoy_index.add_item(idx, emb)
annoy_index.build(10)

True

In [45]:
with open(f'../data/testimonies_complete.npy','wb') as np_file:
        np.save(np_file, doc_embeddings)

In [39]:
annoy_index.save("../data/testimonies_complete.ann")

True

In [46]:
# doc_embeddings.savez("../data/testimonies_complete")